In [2]:
!pip install -q langchain langchain-community langchain-openai
!pip install -q yfinance pandas numpy scikit-learn
!pip install -q faiss-cpu newspaper3k transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""

TICKER_MAP = {
    "apple": "AAPL",
    "apple inc": "AAPL",
    "microsoft": "MSFT",
    "microsoft corp": "MSFT",
    "google": "GOOGL",
    "alphabet": "GOOGL",
    "amazon": "AMZN",
    "tesla": "TSLA"
}


In [4]:
def resolve_ticker(query: str):
    q = query.lower()
    for name, ticker in TICKER_MAP.items():
        if name in q:
            return ticker
    return None

def detect_intent(query: str):
    q = query.lower()
    if any(w in q for w in ["why", "cause", "reason"]):
        return "explain_move"
    if any(w in q for w in ["when", "trend", "go up", "increase"]):
        return "trend_analysis"
    return "general"


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

def get_price_data(ticker, period="6mo"):
    df = yf.download(ticker, period=period, progress=False)
    df["return"] = df["Close"].pct_change()
    return df.dropna()

def detect_trend(df, window=10):
    df["ma"] = df["Close"].rolling(window).mean()
    df["trend"] = df["Close"] > df["ma"]

    start = None
    for i in range(1, len(df)):
        if df["trend"].iloc[i] and not df["trend"].iloc[i - 1]:
            start = df.index[i]
            break

    return {
        "trend_start": str(start),
        "price_change_pct": round(
            (df["Close"].iloc[-1] / df["Close"].iloc[0] - 1) * 100, 2
        )
    }

def detect_drop(df, threshold=-0.03):
    drops = df[df["return"] < threshold]
    if drops.empty:
        return None

    last = drops.iloc[-1]
    return {
        "date": str(last.name),
        "drop_pct": round(last["return"] * 100, 2)
    }


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

SAMPLE_NEWS = [
    "The company reported weaker-than-expected earnings and cut revenue guidance.",
    "Investors reacted negatively to slowing growth in core business segments.",
    "The broader technology sector declined amid macroeconomic uncertainty."
]

def build_vector_db(texts):
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = splitter.create_documents(texts)
    return FAISS.from_documents(docs, embeddings)

vector_db = build_vector_db(SAMPLE_NEWS)

def search_news(query, k=3):
    results = vector_db.similarity_search(query, k=k)
    return [r.page_content for r in results]

In [ ]:
def correlate_events(price_event, news_chunks):
    correlations = []
    for text in news_chunks:
        score = 0
        if "earnings" in text.lower():
            score += 0.4
        if "guidance" in text.lower():
            score += 0.3
        if "growth" in text.lower():
            score += 0.2

        if score > 0:
            correlations.append({
                "event": text[:150],
                "confidence": round(score, 2)
            })
    return correlations


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.2)

def synthesize_answer(query, ticker, price_event, news, correlations):
    prompt = f"""
You are a financial analyst.

User question: {query}
Ticker: {ticker}

Price event:
{price_event}

Relevant news:
{news}

Correlated explanations:
{correlations}

Explain the market movement clearly and concisely.
Avoid speculation.
"""
    return llm.predict(prompt)


In [ ]:
def handle_query(query):
    ticker = resolve_ticker(query)
    if not ticker:
        return " Could not identify the company."

    intent = detect_intent(query)
    df = get_price_data(ticker)

    if intent == "trend_analysis":
        trend = detect_trend(df)
        return (
            f" {ticker} started trending up around {trend['trend_start']} "
            f"with a {trend['price_change_pct']}% move."
        )

    if intent == "explain_move":
        drop = detect_drop(df)
        news = search_news(f"{ticker} earnings guidance")
        correlations = correlate_events(drop, news)

        return synthesize_answer(
            query, ticker, drop, news, correlations
        )

    return "Ask me why a stock moved or when it started trending."


In [ ]:
while True:
    q = input("Ask> ")
    if q.lower() in ["exit", "quit"]:
        break
    print(handle_query(q))
